# incremental_enrollment-score_normalization
-------------

z-norm과 s-norm을 구현하였다.

사용된 trial embed와 언제 enroll이 일어났는지 정보가 필요하다.

In [1]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import pandas as pd
import pickle
import numpy as np
import sys
import os

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


/opt/conda/envs/pytorch-py3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import itertools
import torch
from torch.nn.functional import cosine_similarity
from plot_ROC import plot_ROC

In [3]:
sys.path.append('/host/projects/sv_experiments/sv_system/')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
def key2df(keys, delimeter="-"):
    key_df = pd.DataFrame(keys, columns=['key'])
    key_df['spk'] = key_df.key.apply(lambda x: x.split(delimeter)[0])
    key_df['label'] = key_df.groupby('spk').ngroup()
    key_df['origin'] = key_df.spk.apply(lambda x: 'voxc2' if x.startswith('id') else 'voxc1')
    key_df = key_df.set_index('key')
    
    return key_df

In [5]:
keys = pickle.load(open("./voxc2_fbank64_embeds/sv_keys.pkl", "rb"))
key_df = key2df(keys)
key2id = {k:v for v, k in enumerate(keys)}

In [6]:
def uttrkey_to_id(uttr_keys):
    return [key2id[key] for key in uttr_keys]

In [7]:
embeds_precise = torch.from_numpy(np.load("./voxc2_fbank64_precise_untied_embeds/sv_embeds.npy"))

## Parsing trace then score normalization

In [8]:
base_dir = "./figs/untiedPrecise_untiedPrecise_fixed_length_pos40/700f_50f/meanCos_posNeg_v0/"
enroll_id_sets = pickle.load(open(base_dir+"enroll_id_sets.pkl", "rb"))
trials_id_sets = pickle.load(open(base_dir+"trials_id_sets.pkl", "rb"))
trials_label_sets = pickle.load(open(base_dir+"trials_label_sets.pkl", "rb"))
result_df = pd.read_pickle(base_dir + "result_df.pkl")

## Imposter set

In [9]:
validation_trial = pickle.load(open("./cases/fixed_length_pos40/700f_50f/validation/val_thresh_trials.pkl", 'rb'))
imposter_set = validation_trial.enroll_idx.unique()[:10000]
imposter_tensor = embeds_precise[uttrkey_to_id(imposter_set)]

In [10]:
n_enroll_used = 10

### Cosine score

In [11]:
from inc_utils import compute_eer

cos_base_pos_score_records = []
cos_base_neg_score_records = []

cos_inc_pos_score_records = []
cos_inc_neg_score_records = []

for trial_idx in range(len(enroll_id_sets)):
    enroll_id = enroll_id_sets[trial_idx]
    enroll_spk = key_df.iloc[enroll_id].spk
    trials_id = trials_id_sets[trial_idx]
    trials_label = trials_label_sets[trial_idx]
    trial_df = result_df[(result_df.spk == enroll_spk.values[0]) & (result_df.n_neg == len(trials_id)-40)]
    enroll_timestamp = np.nonzero(np.array(trial_df.inc_enroll.values[0]) > -1)

    init_enroll_tensor = embeds_precise[enroll_id]
    test_tensor = embeds_precise[trials_id]

    inc_enroll_tensor = test_tensor[enroll_timestamp]
    all_enroll_tensor = torch.cat([init_enroll_tensor, inc_enroll_tensor], dim=0)

    base_trial_score = cosine_similarity(init_enroll_tensor.unsqueeze(1), 
                                         test_tensor.unsqueeze(0), dim=2).numpy()
    
    inc_trial_score = cosine_similarity(all_enroll_tensor.unsqueeze(1), 
                                     test_tensor.unsqueeze(0), dim=2).numpy()
        
#     trial_len = len(trials_id)
#     n_enroll = all_enroll_tensor.shape[0]
#     t_stamps = [-1] + enroll_timestamp[0].tolist() + [trial_len-1]
# #     t_stamps.reverse()
# # =================== base ===================
#     base_pos_score_records += [base_trial_score[:, trials_label == 1].mean(0)]
#     base_neg_score_records += [base_trial_score[:, trials_label == 0].mean(0)]
    
# # =================== inc ===================
#     trial_score_ = np.zeros(trial_len,)
#     for i in range(len(t_stamps)-1):
# #         print(t_stamps[i]+1, t_stamps[i+1]+1)
# #         print(max(0, (i+1)-10), i+1)
#         trial_score_[t_stamps[i]+1:t_stamps[i+1]+1] = \
#         inc_trial_score[max(0, (i+1)-10):i+1,t_stamps[i]+1:t_stamps[i+1]+1].mean(0)
        
#     inc_pos_score_records += [trial_score_[trials_label == 1]]
#     inc_neg_score_records += [trial_score_[trials_label == 0]]


    trial_len = len(trials_id)
    n_enroll = all_enroll_tensor.shape[0]
    t_stamps = enroll_timestamp[0].tolist() + [trial_len]
    t_stamps.reverse()
# =================== base ===================
    cos_base_pos_score_records += [base_trial_score[:, trials_label == 1].mean(0)]
    cos_base_neg_score_records += [base_trial_score[:, trials_label == 0].mean(0)]
    
# =================== inc ===================
    trial_score_ = np.zeros(trial_len,)
    for i, t in enumerate(t_stamps):
        trial_score_[:t] = inc_trial_score[max(0, (n_enroll-i)-n_enroll_used):n_enroll-i,:t].mean(0)
#         trial_score_[:t] = inc_trial_score[0:n_enroll-i,:t].mean(0)
        
    cos_inc_pos_score_records += [trial_score_[trials_label == 1]]
    cos_inc_neg_score_records += [trial_score_[trials_label == 0]]

### Z-norm

In [12]:
def z_scores(imposter_scores, scores):
    z_mu = imposter_scores.mean(dim=1, keepdim=True)
    z_std = imposter_scores.std(dim=1, keepdim=True)
    
    return (scores - z_mu) / z_std

In [13]:
from inc_utils import compute_eer

z_base_pos_score_records = []
z_base_neg_score_records = []

z_inc_pos_score_records = []
z_inc_neg_score_records = []

for trial_idx in range(len(enroll_id_sets)): 
    enroll_id = enroll_id_sets[trial_idx]
    enroll_spk = key_df.iloc[enroll_id].spk
    trials_id = trials_id_sets[trial_idx]
    trials_label = trials_label_sets[trial_idx]
    trial_df = result_df[(result_df.spk == enroll_spk.values[0]) & (result_df.n_neg == len(trials_id)-40)]
    enroll_timestamp = np.nonzero(np.array(trial_df.inc_enroll.values[0]) > -1)

    init_enroll_tensor = embeds_precise[enroll_id]
    test_tensor = embeds_precise[trials_id]

    inc_enroll_tensor = test_tensor[enroll_timestamp]
    all_enroll_tensor = torch.cat([init_enroll_tensor, inc_enroll_tensor], dim=0)

    # z_score
    base_z_imp_scores = cosine_similarity(init_enroll_tensor.unsqueeze(1), 
                                     imposter_tensor.unsqueeze(0), dim=2)
    base_trial_score = cosine_similarity(init_enroll_tensor.unsqueeze(1), 
                                         test_tensor.unsqueeze(0), dim=2)
    base_trial_score = z_scores(base_z_imp_scores, base_trial_score).numpy()
    
    inc_z_imp_scores = cosine_similarity(all_enroll_tensor.unsqueeze(1), 
                                     imposter_tensor.unsqueeze(0), dim=2)
    inc_trial_score = cosine_similarity(all_enroll_tensor.unsqueeze(1), 
                                     test_tensor.unsqueeze(0), dim=2)
    inc_trial_score = z_scores(inc_z_imp_scores, inc_trial_score).numpy()


    trial_len = len(trials_id)
    n_enroll = all_enroll_tensor.shape[0]
    t_stamps = enroll_timestamp[0].tolist() + [trial_len]
    t_stamps.reverse()
# =================== base ===================
    z_base_pos_score_records += [base_trial_score[:, trials_label == 1].mean(0)]
    z_base_neg_score_records += [base_trial_score[:, trials_label == 0].mean(0)]
    
# =================== inc ===================
    trial_score_ = np.zeros(trial_len,)
    for i, t in enumerate(t_stamps):
        trial_score_[:t] = inc_trial_score[max(0, (n_enroll-i)-n_enroll_used):n_enroll-i,:t].mean(0)
#         trial_score_[:t] = inc_trial_score[0:n_enroll-i,:t].mean(0)
        
    z_inc_pos_score_records += [trial_score_[trials_label == 1]]
    z_inc_neg_score_records += [trial_score_[trials_label == 0]]

### s-norm

In [14]:
u_imp_tensor = imposter_tensor / imposter_tensor.norm(dim=1, keepdim=True)
imp_mean = u_imp_tensor.mean(dim=0, keepdim=True)
imp_dig_cov = torch.std(u_imp_tensor, dim=0)

In [15]:
# normalized cos similarity
def s_scores(target_tensor, test_tensor):   
    u_target = target_tensor / target_tensor.norm(dim=1, keepdim=True) 
    u_test = test_tensor / test_tensor.norm(dim=1, keepdim=True)

    target_mean = (u_target - imp_mean)
    test_mean = (u_test - imp_mean)
    target_std = u_target.matmul(imp_dig_cov)
    test_std = u_test.matmul(imp_dig_cov)
    
    s_score = target_mean.matmul(test_mean.t())\
                        / target_std.view(-1,1)\
                        / test_std.view(1,-1)
    
    return s_score

In [16]:
from inc_utils import compute_eer

s_base_pos_score_records = []
s_base_neg_score_records = []

s_inc_pos_score_records = []
s_inc_neg_score_records = []

for trial_idx in range(len(enroll_id_sets)):
    enroll_id = enroll_id_sets[trial_idx]
    enroll_spk = key_df.iloc[enroll_id].spk
    trials_id = trials_id_sets[trial_idx]
    trials_label = trials_label_sets[trial_idx]
    trial_df = result_df[(result_df.spk == enroll_spk.values[0]) & (result_df.n_neg == len(trials_id)-40)]
    enroll_timestamp = np.nonzero(np.array(trial_df.inc_enroll.values[0]) > -1)

    init_enroll_tensor = embeds_precise[enroll_id]
    test_tensor = embeds_precise[trials_id]

    inc_enroll_tensor = test_tensor[enroll_timestamp]
    all_enroll_tensor = torch.cat([init_enroll_tensor, inc_enroll_tensor], dim=0)

    base_trial_score = s_scores(init_enroll_tensor, test_tensor).numpy()
    inc_trial_score = s_scores(all_enroll_tensor, test_tensor).numpy()
        
    trial_len = len(trials_id)
    n_enroll = all_enroll_tensor.shape[0]
    t_stamps = enroll_timestamp[0].tolist() + [trial_len]
    t_stamps.reverse()
# =================== base ===================
    s_base_pos_score_records += [base_trial_score[:, trials_label == 1].mean(0)]
    s_base_neg_score_records += [base_trial_score[:, trials_label == 0].mean(0)]
    
# =================== inc ===================
    trial_score_ = np.zeros(trial_len,)
    for i, t in enumerate(t_stamps):
        trial_score_[:t] = inc_trial_score[max(0, (n_enroll-i)-n_enroll_used):n_enroll-i,:t].mean(0)
#         trial_score_[:t] = inc_trial_score[0:n_enroll-i,:t].mean(0)
        
    s_inc_pos_score_records += [trial_score_[trials_label == 1]]
    s_inc_neg_score_records += [trial_score_[trials_label == 0]]
    

In [17]:
for idx in range(0,40,10):
    print(compute_eer(np.concatenate(cos_base_pos_score_records[idx:idx+10], axis=0),
            np.concatenate(cos_base_neg_score_records[idx:idx+10], axis=0)))
for idx in range(0,40,10):
    print(compute_eer(np.concatenate(z_base_pos_score_records[idx:idx+10], axis=0),
                    np.concatenate(z_base_neg_score_records[idx:idx+10], axis=0)))
for idx in range(0,40,10):
    print(compute_eer(np.concatenate(s_base_pos_score_records[idx:idx+10], axis=0),
                    np.concatenate(s_base_neg_score_records[idx:idx+10], axis=0)))

0.03
0.0275
0.0297
0.0295
0.02
0.025
0.0269
0.027
0.0
0.015
0.0225
0.0225


In [18]:
for idx in range(0,40,10):
    print(compute_eer(np.concatenate(cos_inc_pos_score_records[idx:idx+10], axis=0),
            np.concatenate(cos_inc_neg_score_records[idx:idx+10], axis=0)))
for idx in range(0,40,10):
    print(compute_eer(np.concatenate(z_inc_pos_score_records[idx:idx+10], axis=0),
            np.concatenate(z_inc_neg_score_records[idx:idx+10], axis=0)))
for idx in range(0,40,10):
    print(compute_eer(np.concatenate(s_inc_pos_score_records[idx:idx+10], axis=0),
            np.concatenate(s_inc_neg_score_records[idx:idx+10], axis=0)))

0.01
0.0075
0.0114
0.0125
0.01
0.0075
0.0125
0.0104
0.0
0.0025
0.005
0.0045
